In [ ]:
import numpy as np

In [ ]:
import optimization.optim_tools as optim_tools

kwargs = optim_tools.bisect_max(0, 1, None, None, None,
           bisection_tol=1e-3, solver=None, bisect_verbose=False, max_iters=1100, warm_start=True)
kwargs['max_iters'] = 2000
kwargs

In [ ]:
def svd_inverse(A):
    """ Inverts a matrix using singular value decomposition (SVD).
        The matrix doesn't have to be square or of full rank.
    """
    # A = U*(I*s)*V => is easier to invert (see literature)
    U, s, V = np.linalg.svd(A, full_matrices=False)  # SVD decomposition of A
    for i in xrange(0, len(s)):
        if s[i] > 0.001:
            s[i] = 1/s[i]        # ToDo: damping near singularities
    return np.dot(np.dot(V.T, np.diag(s)), U.T)

In [ ]:
def svd_solve(A, b):
    U, s, V = np.linalg.svd(A, full_matrices=False)  # SVD decomposition of A
    for i in xrange(0, len(s)):
        if s[i] > 0.001:
            s[i] = 1/s[i]        # ToDo: damping near singularities
    return V.T.dot(np.diag(s).dot(U.T.dot(b)))

In [ ]:
def svd_solve2(A, b):
    U, s, V = np.linalg.svd(A, full_matrices=False)  # SVD decomposition of A
    s[s > 0.001] = 1/s[s > 0.001]
    return V.T.dot(np.diag(s).dot(U.T.dot(b)))

In [ ]:
n=6
m=7
A_m = np.random.rand(n,m)
y_m = np.random.rand(n)
np.random.seed(0)

In [ ]:
%%timeit

A_m_inv = svd_inverse(A_m)
dq = np.dot(A_m_inv, y_m)

In [ ]:
%%timeit

dq = svd_solve(A_m, y_m)

In [ ]:
%%timeit

dq = svd_solve2(A_m, y_m)

In [ ]:
#import numpy as np
#import control as con
#from numpy import linalg as LA

import cvxpy
#import optimization.optim_tools as optim_tools#own file with helper

In [ ]:
import numpy as np
import cvxpy

n = 3

A = np.matrix([[0,   1,       0],
               [-10, -1.167, 25],
               [0,   0,    -0.8]])

P = cvxpy.Semidef(n, n) # Implys (459) (semidefinite for numerical reasons?)

constraints = [P >> 1,
               A.T*P + P*A << 0]

In [ ]:
import smcp

# Objective representation
obj = cvxpy.Minimize(0) # Identical to geo_mean (in term of convexity and result)
#obj = cvxpy.Maximize(cvxpy.log_det(Q)) # Identical to geo_mean (in term of convexity and result)
#obj = cvxpy.Minimize(-cvxpy.geo_mean(Q)) # Not yet implemented

prob = cvxpy.Problem(obj, constraints)
prob.solve(solver=cvxpy.SCS, verbose=True)